Notebook para limpieza y tratamiento de datos del dataset de indicadores de género en LATAM.
Cada fila del dataset representa una serie temporal asociada a un indicador particular para un país,
con valores anuales desde el año 2000 hasta el 2017.
El objetivo principal es limpiar el dataset eliminando columnas y filas no informativas,
y luego aplicar imputación por regresión lineal sobre las series temporales incompletas.

❗ Esta notebook asume que el archivo df_latam_genero.csv está en la carpeta ./data/clean/parciales/

# 1. Importación de librerías necesarias
Estas librerías permiten la carga y manipulación de datos (pandas, numpy), visualización (matplotlib, seaborn)


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os


# 2. Cargar el dataset y eliminar columnas con más del 70% de valores NaN

En esta etapa, se carga el archivo original 'df_latam_genero.csv' quecontiene la data de latinoamérica filtrada por indicadores asociados a género (ver notebook 01_EDA para más detalles de la categorización de indicadores). Luego, se identifican y eliminan las columnas en las que más del 70% de los datos son NaN (faltantes), lo que indica que esas columnas tienen poca o nula utilidad analítica. Esto permite reducir el ruido en el análisis posterior. Es importante mencionar que, bajo este criterio de eliminación, las columnas correspondientes a los años desde 2018 a 2022 fueron eliminadas por tener demasiados valores faltantes. Esto sugiere que muchos países dejaron de reportar sus indicadores a partir de 2018.


In [6]:
df_latam_genero = pd.read_csv(r'..\data\clean\parciales\df_latam_genero.csv')

# Identify columns with more than 70% NaN values
nan_columns = []
for column in df_latam_genero.columns:
    # Calculate percentage of NaN values
    percentage_nan = (df_latam_genero[column].isnull().sum() / len(df_latam_genero)) * 100

    # Print the results
    print(f"Column '{column}': {percentage_nan:.2f}% NaN values")

    # Check if percentage of NaN values is more than 70%
    if percentage_nan > 70:
        nan_columns.append(column)

# Print the names of the columns to be dropped
print("Dropping the following columns with more than 70% NaN values:")
for column_name in nan_columns:
    print(column_name)

# Drop the identified columns
df_latam_genero_colsdropped = df_latam_genero.drop(columns=nan_columns)
df_latam_genero_colsdropped.to_csv('df_latam_genero_dropped.csv', index=False)

Column 'Country Name': 0.00% NaN values
Column 'Country Code': 0.00% NaN values
Column 'Indicator Name': 0.00% NaN values
Column 'Indicator Code': 0.00% NaN values
Column '2000': 52.53% NaN values
Column '2001': 40.01% NaN values
Column '2002': 38.37% NaN values
Column '2003': 35.91% NaN values
Column '2004': 37.00% NaN values
Column '2005': 37.82% NaN values
Column '2006': 30.30% NaN values
Column '2007': 42.61% NaN values
Column '2008': 42.68% NaN values
Column '2009': 34.27% NaN values
Column '2010': 55.27% NaN values
Column '2011': 45.62% NaN values
Column '2012': 33.65% NaN values
Column '2013': 65.05% NaN values
Column '2014': 54.24% NaN values
Column '2015': 37.41% NaN values
Column '2016': 61.22% NaN values
Column '2017': 49.93% NaN values
Column '2018': 85.50% NaN values
Column '2019': 86.94% NaN values
Column '2020': 95.21% NaN values
Column '2021': 80.71% NaN values
Column '2022': 100.00% NaN values
Column 'region': 0.00% NaN values
Column 'Categoria Indicador': 0.00% NaN va

# 3. Eliminar filas con series de tiempo completamente vacías
En esta etapa, los valores numéricos de cada fila representa una serie de tiempo: los valores de un indicador específico para un país,
a lo largo de los años 2000 al 2017. Si una fila no contiene ningún valor numérico en este rango de años,
significa que no se tiene información útil sobre esa serie, por lo que se elimina. De nuevo, esto significa que un país no reportó un indicador a lo largo de todos los años de medición de este dataset. Se resalta aquí el caso particular de Venezuela cuya gran mayoría de indicadores fue rechazado en esta etapa



In [ ]:
import pandas as pd

# Asumiendo que df_latam_genero ya está definido

# Seleccionar las columnas de años
year_cols = [str(year) for year in range(2000, 2018)]  # Asegúrate de que el rango sea correcto

# Identificar las filas con todos los valores NaN en las columnas de años
rows_to_drop = df_latam_genero_colsdropped[df_latam_genero_colsdropped[year_cols].isnull().all(axis=1)]

# Imprimir el país e indicador de las filas que se van a eliminar
for index, row in rows_to_drop.iterrows():
    print(f"Eliminando fila: País - {row['Country Name']}, Indicador - {row['Indicator Name']}")

# Eliminar las filas identificadas
df_latam_genero_colsdropped_rowsdropped = df_latam_genero_colsdropped.drop(rows_to_drop.index)

df_latam_genero_colsdropped_rowsdropped.to_csv('df_latam_genero_colsdropped_rowsdropped.csv', index=False)

Eliminando fila: País - Argentina, Indicador - Females, as a share of private paid employees by occupational group: Clerks
Eliminando fila: País - Argentina, Indicador - Females, as a share of public paid employees by occupational group: Clerks
Eliminando fila: País - Argentina, Indicador - Public sector female employment, as a share of paid employment by occupational group: Clerks
Eliminando fila: País - Bolivia, Indicador - Gender wage premium in the public sector, by industry: Social Security (compared to male paid employees)
Eliminando fila: País - Bolivia, Indicador - P-Value: Gender wage premium in the public sector, by industry: Social Security (compared to male paid employees)
Eliminando fila: País - Brazil, Indicador - Females, as a share of public paid employees by industry: Core Public Administration
Eliminando fila: País - Brazil, Indicador - Females, as a share of public paid employees by industry: Public Safety
Eliminando fila: País - Brazil, Indicador - Females, as a sha

# 4. Imputación de valores faltantes usando regresión lineal por fila (serie temporal)
Los valores numéricos de cada fila del dataset (i.e., columnas 2000, 2001, etc) contiene una serie temporal para un país e indicador.
Si la serie tiene al menos dos valores conocidos, se aplica una regresión lineal simple
para predecir los valores faltantes. Si hay menos de dos valores, no es posible aplicar regresión, por lo que la fila se elimina. Esta imputación permite completar datos de forma coherente
manteniendo las tendencias temporales observadas.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Cargar el archivo CSV
#df_latam_genero_colsdropped_rowsdropped = pd.read_csv('df_latam_genero_colsdropped_rowsdropped.csv')

# Definir las columnas correspondientes a los años (2000 a 2017)
years = list(range(2000, 2018))
year_cols = [str(year) for year in years]

# Identificar filas que tengan al menos 2 valores no faltantes en la serie temporal
mask_valid = df_latam_genero_colsdropped_rowsdropped[year_cols].count(axis=1) >= 2

# Filtrar las filas a descartar (menos de 2 datos válidos) e imprimir país e indicador
df_invalid = df_latam_genero_colsdropped_rowsdropped[~mask_valid]
for idx, row in df_invalid.iterrows():
    print(f"Fila descartada - País: {row['Country Name']}, Indicador: {row['Indicator Name']}")

# Conservar únicamente las filas válidas para imputar
df_valid = df_latam_genero_colsdropped_rowsdropped[mask_valid].copy()

def imputar_fila(row):
    # Extraer la serie temporal para la fila
    ts = row[year_cols]
    ts_valid = ts.dropna()

    # Convertir los años (índices) a un array de enteros para el entrenamiento
    X_train = np.array([int(year) for year in ts_valid.index]).reshape(-1, 1)
    y_train = ts_valid.values.astype(float)

    # Ajustar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Para cada año con dato faltante, predecir el valor y asignarlo
    for col in ts.index[ts.isna()]:
        year_int = int(col)
        pred = model.predict(np.array([[year_int]]))[0]
        row[col] = pred

    return row

# Aplicar la función de imputación a cada fila válida
df_latam_genero_colsdropped_rowsdropped_imputed = df_valid.apply(imputar_fila, axis=1)

# Guardar el DataFrame resultante en un nuevo archivo CSV
df_latam_genero_colsdropped_rowsdropped_imputed.to_csv('df_latam_genero_colsdropped_rowsdropped_imputed.csv', index=False)

print("Imputación completada y archivo guardado.")

columnas_con_nan = df_latam_genero_colsdropped_rowsdropped_imputed.columns[df_latam_genero_colsdropped_rowsdropped_imputed.isnull().any()].tolist()
print("Columnas con al menos un NaN:")
for columna in columnas_con_nan:
    print(columna)
if not columnas_con_nan:
    print("No hay columnas con NaN.")

Fila descartada - País: Bolivia, Indicador: Females, as a share of public paid employees by industry: Social Security
Fila descartada - País: Guatemala, Indicador: Gender wage premium in the private sector, by occupation: Medical workers (compared to male paid employees)
Fila descartada - País: Guatemala, Indicador: Gender wage premium in the private sector, by occupation: Teachers (compared to male paid employees)
Fila descartada - País: Guatemala, Indicador: Public sector wage premium for females, by occupation: Medical workers (compared to paid wage employees)
Fila descartada - País: Guatemala, Indicador: Public sector wage premium for females, by occupation: Teachers (compared to paid wage employees)
Fila descartada - País: Guatemala, Indicador: P-Value: Gender wage premium in the private sector, by occupation: Medical workers (compared to male paid employees)
Fila descartada - País: Guatemala, Indicador: P-Value: Gender wage premium in the private sector, by occupation: Teachers (

# 5. Reorganizar los datos por país e indicadores para un año específico
Después de imputar los valores faltantes, puede ser útil reorganizar la estructura del dataset para análisis por año. La función indicadores_por_pais_y_año transforma el DataFrame original, que tiene múltiples filas por país (una por indicador), en un formato más compacto en el que cada país tiene una sola fila, y cada indicador se convierte en una columna. Esta forma es útil para análisis multivariado o clustering por país.

In [ ]:
def indicadores_por_pais_y_año(df, año):
  """
  Crea un DataFrame con los países como filas, Country Code como índice, y los indicadores como columnas para un año específico.

  Args:
    df: El DataFrame original (df_latam_genero_colsdropped_rowsdropped_imputed).
    año: El año para el que se quiere obtener los indicadores.

  Returns:
    Un nuevo DataFrame con los países como filas, Country Code como índice, y los indicadores como columnas.
  """

  # 1. Filtrar el DataFrame por el año especificado
  df_año = df[['Country Code', 'Country Name', 'Indicator Name', str(año)]]

  # 2. Pivotar el DataFrame para obtener la estructura deseada
  df_pivoteado = df_año.pivot(index='Country Code', columns='Indicator Name', values=str(año))

  #3. Restablecer el nombre de las columnas
  df_pivoteado.columns.name = None

  #4. Agregar 'Country Name' como columna
  df_pivoteado.insert(0, 'Country Name', df_año.groupby('Country Code')['Country Name'].first())

  return df_pivoteado

df_pivoteado = indicadores_por_pais_y_año(df_latam_genero_colsdropped_rowsdropped_imputed, 2012)
#